In [3]:
import numpy as np
import xarray as xr
import gc

import matplotlib.pyplot as plt
import matplotlib as mpl
%matplotlib inline
import cartopy.crs as ccrs
import cartopy.feature as cfeature

In [5]:
path='E:\\CESM-Dryland\\'

In [ ]:
def make_Ramp(ramp_colors,plot_cbar): 
    from colour import Color
    from matplotlib.colors import LinearSegmentedColormap
    
    color_ramp = LinearSegmentedColormap.from_list('my_list',[ Color( c1 ).rgb for c1 in ramp_colors ] )
    if plot_cbar==True:
        plt.figure( figsize = (15,3))
        plt.imshow( [list(np.arange(0, len( ramp_colors) , 0.1)) ] , interpolation='nearest', origin='lower', cmap= color_ramp )
        plt.xticks([])
        plt.yticks([])
    return color_ramp

OrBu = make_Ramp(['#e17701','#9af764','grey','#02ccef','#056eee',],plot_cbar=False)
grey = make_Ramp(['grey','grey',],plot_cbar=False)

In [ ]:
def get_ds_varname(ds):
    vars = list(ds.keys())
    if len(vars)==1:
        var_name=vars[0]
    else:
        print('There should is more than ONE variables in the each dataset')
    return var_name

def get_variable_name(DataArray):
    return DataArray.name
    
def is_increasing(a):
    diff = np.ediff1d(a)
    return np.all(diff>0)
    
def flip_lat_lon(ds):
    # Convert extent
    # Check latitudes from -90-90
    if is_increasing(ds.lat)==False:
        print('flip lat')
        temp = ds.reindex(lat=list(reversed(ds.lat)))
        ds = temp.copy()
    # Check longituted from 0-360
    if np.any(ds.lon<0)==True:
        print('flip lon')
        lon_fill = xr.where(ds['lon']<0,ds['lon']+360,ds['lon'])
        ds_360   = ds.assign_coords({'lon':lon_fill%360})
        temp     = ds_360.sel(**{'lon':sorted(ds_360['lon'])})
        ds       = temp.copy()
    return ds
    
def open_ai():
    ds = xr.open_dataset(path+'ai_plot_mask.nc')
    varname =  get_ds_varname(ds)
    out = flip_lat_lon(ds[varname])
    ds.close()
    return out